This notebook is for trimming silent beginning/endings on mp3 files

In [6]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
from tqdm import tqdm
import os
from functools import reduce

# Define the directories
input_dir = r'..\data\audio_files\raw_house'  # Directory to raw mp3 files
output_dir = r'..\data\audio_files\processed'  # Directory to export trimmed mp3 files

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to strip silence
def strip_silence(sound, silence_thresh=-50):
    nonsilent_ranges = detect_nonsilent(sound, min_silence_len=1000, silence_thresh=silence_thresh)
    # Concatenate the nonsilent portions of the audio
    return reduce(lambda acc, val: acc + sound[val[0]:val[1]], nonsilent_ranges, AudioSegment.empty()) if nonsilent_ranges else sound

# Process all audio files in the input directory
for filename in tqdm([f for f in os.listdir(input_dir) if f.endswith('.mp3')], desc="Processing audio files", unit="file"):
    file_path = os.path.join(input_dir, filename)
    output_file_path = os.path.join(output_dir, filename)
    
    # Skip processing if the file already exists in the output directory
    if os.path.exists(output_file_path):
        print(f"File {filename} already processed. Skipping.")
        continue
    
    # Load the audio file and strip silence
    sound = AudioSegment.from_file(file_path)
    stripped_sound = strip_silence(sound)
    
    # Export the stripped audio without metadata
    stripped_sound.export(output_file_path, format='mp3')

Processing audio files: 100%|████████████████████████████████████████████████████| 364/364 [2:02:11<00:00, 20.14s/file]
